## Play around datasets and create one with hierarchical prior


In [12]:
import os
import sys
from tensorflow.compat.v1 import gfile

sys.path.append(os.path.abspath("/home/mwu34/disentanglement_lib"))
import numpy as np
import pandas as pd

DSPRITES_PATH = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites",
    "dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz"
)

In [7]:
with gfile.Open(DSPRITES_PATH, "rb") as data_file:
    # Data was saved originally using python2, so we need to set the encoding.
    data = np.load(data_file, encoding="latin1", allow_pickle=True)
    images = np.array(data["imgs"])  # has shape 737280 * 64 * 64
    factor_sizes = np.array(data["metadata"][()]["latents_sizes"], dtype=np.int64)

In [14]:
from disentanglement_lib.data.ground_truth.util import CorrelatedSplitDiscreteStateSpace


correlated_space = CorrelatedSplitDiscreteStateSpace(factor_sizes, [3, 4])

RuntimeError: Required bindings for `correlation_details` not provided in config: ['corr_indices', 'corr_type']

In [39]:
# trying to get mapping from specific parameter combinations to model num
from disentanglement_lib.config.unsupervised_study_v1.sweep import get_config
import pandas as pd

configs = get_config()
df = pd.DataFrame(configs)

In [3]:
len(df)

54

In [9]:
# trying to get mapping from specific parameter combinations to model num
# from disentanglement_lib.config.unsupervised_study_v1.sweep import get_config
from disentanglement_lib.config.double_descent_study_v2.sweep import get_config
import pandas as pd

configs = get_config()
df = pd.DataFrame(configs)

# Get the beta_tc_vae models using dsprites and a beta param of 4.
model = df['model.name'] == "beta_vae"
dataset = df['dataset.name'] == "dsprites_full"
# beta = df["vae.beta"] == 1.0

# Give all the possible model configs unused parameters excluded (e.g., annealed_vae.gamma)
# print(df[model & dataset & beta].dropna(axis=1))

# Give the list of model ids with this configuration
df[model & dataset].head(200)


,correlation_details.corr_indices,dataset.name,correlation_hyperparameter.line_width,correlation_details.corr_type,encoder.encoder_fn,decoder.decoder_fn,correlation.active_correlation,model.name,vae.beta,model.model,conv_encoder.num_parameters_scale,deconv_decoder.num_parameters_scale,model.random_seed
0,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.03125,0.03125,0
1,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.06250,0.06250,0
2,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.09375,0.09375,0
3,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.12500,0.12500,0
4,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.15625,0.15625,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,8.000,@vae(),0.87500,0.87500,0
92,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,8.000,@vae(),0.90625,0.90625,0
93,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,8.000,@vae(),0.93750,0.93750,0
94,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,8.000,@vae(),0.96875,0.96875,0


In [13]:
from disentanglement_lib.data.ground_truth import named_data

dsprites = named_data.get_named_ground_truth_data('dsprites_full')

In [16]:
dsprites.factor_sizes

array([ 1,  3,  6, 40, 32, 32])

In [17]:
dsprites.full_factor_sizes

[1, 3, 6, 40, 32, 32]

In [18]:
dsprites.factor_bases

array([7.3728e+05, 2.4576e+05, 4.0960e+04, 1.0240e+03, 3.2000e+01,
       1.0000e+00])

In [26]:
random_state = np.random.RandomState(0)
dsprites.sample_observations(10, random_state).shape

(10, 64, 64, 1)

In [54]:
# split dsprites dataset into two parts

full_size = np.prod(dsprites.factor_sizes)
train_dataset_size = 700000
train_indices = np.random.choice(full_size, train_dataset_size, replace=False)
train_indices = np.sort(train_indices)
valid_indices = np.delete(np.arange(full_size), train_indices)

train_indices_path = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites", f"train700k.npy"
)
valid_indices_path = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites", f"valid700k.npy"
)
np.save(train_indices_path, train_indices)
np.save(valid_indices_path, valid_indices)

In [61]:
gg = np.random.choice(valid_indices, 3, replace=True)
gg

array([467061,   4688, 580110])

In [62]:
4688 in valid_indices

True